### preprocessing

In [ ]:
import pickle

with open("data/purchase.pickle", 'rb') as p:
    purchase = pickle.load(p)

# 아이템 매칭 여부에 관계없이 리뷰 합치기
df = purchase.groupby('리뷰자')['본문'].sum().reset_index()

In [ ]:
# 이스케이프 시퀀스 삭제 : \n 제외
df['본문'] = df['본문'].replace('\xa0', ' ', regex=True)
#특수문자 변경 : !, ^, _을 .으로
df['본문'] = df['본문'].replace(r'\!', '.', regex=True).replace(r'\^', '.', regex=True).replace(r'\_', '.', regex=True)
# .을 \n으로 변경
df['본문'] = df['본문'].replace(r'\.', '\n', regex=True)
# ㅋㅋㅋ ㅎㅎㅎ 등은 \n으로
df['본문'] = df['본문'].replace(r'[ㄱ-ㅎ]', '\n', regex=True)
# 기타 특수문자 제거
df['본문'] = df['본문'].replace(r'[^0-9가-힣a-zA-Z\s\n]', '', regex=True)

In [4]:
df.to_csv("textrank_all item.csv", encoding='utf-8-sig', index=False)

### textrank 알고리즘
- gensim은 한국어를 지원해주지 않음 -> 길이가 짧을 경우 요약 안 됨

In [25]:
%pip install textrankr

     ---------------------------------------- 1.9/1.9 MB 30.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [40]:
%pip install lexrankr

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sklearn: filename=sklearn-0.0.post5-py3-none-any.whl size=2970 sha256=10215e2257e39beec1f2bccdd2f5873af093c5c2e446d24629fa377386766240
  Stored in directory: c:\users\a\appdata\local\pip\cache\wheels\b2\af\1b\ac28f3fb36a8428e3089acdd913e9ee1808e781e3ff6ce2929
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


In [3]:
from textrankr import TextRank
from lexrankr import LexRank
from typing import List
import pandas as pd
import numpy as np

In [4]:
df1 = pd.read_csv("textrank_all item.csv")

# 결측값은 제외하고 요약
df1 = df1.dropna().reset_index(drop=True)

In [5]:
# -어요, -네요, -습니다, -아요 뒤에 \n 삽입
for i,j in enumerate(df1['본문']):
    string = ['어요', '네요', '습니다', '아요']
    text = j
    for k in string:
        idx = text.find(k)
        if idx != -1:
            text = text[:idx+len(k)] + '\n' + text[idx+len(k):]
        else:
            text = text
    df1.loc[i, '본문'] = text

#### textrankr

In [6]:
class MyTokenizer: #리스트 형식으로 저장하기 위한 클래스 선언
    def __call__(self, text:str) -> List[str]:
        tokens: List[str] = text.split()
        return tokens
    
# 리스트 형식으로 저장하는 공간에 데이터들을 요약한다
mytokenizer: MyTokenizer = MyTokenizer()
textrank : TextRank = TextRank(mytokenizer)

In [82]:
df1['textrank1'] = np.nan

for i,j in enumerate(df1['본문']):
    summarized: str = textrank.summarize(j)
    df1.loc[i, 'textrank1'] = summarized

In [83]:
df1.isna().sum() #전부 적용됨

리뷰자          0
본문           0
textrank1    0
dtype: int64

In [84]:
df1[df1['본문'] == df1['textrank1']].shape #요약 전후가 동일한 경우 확인

(188, 3)

In [9]:
help(textrank)

Help on TextRank in module textrankr.textrank object:

class TextRank(builtins.object)
 |  TextRank(tokenizer: Callable[[str], List[str]], tolerance: float = 0.05) -> None
 |  
 |  Args:
 |      tokenizer: a function or a functor of Callable[[str], List[str]] type.
 |      tolerance: a threshold for omitting edge weights.
 |  
 |  Example:
 |      ```
 |      tokenizer: YourTokenizer = YourTokenizer()
 |      textrank: TextRank = TextRank(tokenzier, tolerance=0.05)
 |      summaries: str = textrank.summarize(your_text_here, num_sentences=True, verbose=True)
 |      print(summaries)
 |      ```
 |  
 |  Methods defined here:
 |  
 |  __init__(self, tokenizer: Callable[[str], List[str]], tolerance: float = 0.05) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  summarize(self, text: str, num_sentences: int = 3, verbose: bool = True) -> Union[str, List[str]]
 |      Summarizes the given text, using TextRank algorithm.
 |      
 |      Args:
 |        

In [92]:
help(textrank.summarize)

Help on method summarize in module textrankr.textrank:

summarize(text: str, num_sentences: int = 3, verbose: bool = True) -> Union[str, List[str]] method of textrankr.textrank.TextRank instance
    Summarizes the given text, using TextRank algorithm.
    
    Args:
        text: a raw text to be summarized.
        num_sentences: the number of sentences in a summarization result.
        verbose: if True, it will return a summarized raw text. It will return a summarized list of sentences otherwise.



In [93]:
df1['textrank2'] = np.nan

for i,j in enumerate(df1['본문']):
    summarized: str = textrank.summarize(j, num_sentences=2) # 2줄 요약
    df1.loc[i, 'textrank2'] = summarized

In [94]:
df1.isna().sum() #전부 적용됨

리뷰자          0
본문           0
textrank1    0
textrank2    0
dtype: int64

In [95]:
df1[df1['본문'] == df1['textrank2']].shape #요약 전후가 동일한 경우 확인

(173, 4)

#### lexrankr

In [11]:
class MyTokenizer: #리스트 형식으로 저장하기 위한 클래스 선언
    def __call__(self, text:str) -> List[str]:
        tokens: List[str] = text.split()
        return tokens
    
# 리스트 형식으로 저장하는 공간에 데이터들을 요약한다
mytokenizer: MyTokenizer = MyTokenizer()
lexrank : LexRank = LexRank(mytokenizer)

In [97]:
df1['lexrank1'] = np.nan

for i,j in enumerate(df1['본문']):
    try:
        lexrank.summarize(j)
        summaries: List[str] = lexrank.probe()
        df1.loc[i, 'lexrank1'] = ' '.join(summaries)
    except ValueError:
        text = j.replace("\n", " ")
        lexrank.summarize(text)
        summaries: List[str] = lexrank.probe()
        df1.loc[i, 'lexrank1'] = ' '.join(summaries)

In [98]:
df1.isna().sum()

리뷰자          0
본문           0
textrank1    0
textrank2    0
lexrank1     0
dtype: int64

In [99]:
df1[df1['본문'] == df1['lexrank1']].shape #요약 전후가 동일한 경우 확인

(123, 5)

In [12]:
help(lexrank)

Help on LexRank in module lexrankr.lexrankr object:

class LexRank(builtins.object)
 |  LexRank(tokenizer: Callable[[str], List[str]], similarity_method: str = 'tfidf_cosine', similarity_decay_window: int = 20, similarity_decay_alpha: float = 0.25, similarity_matrix_diagonal_smoothing: bool = False, clustering_method: Union[str, NoneType] = 'birch', birch_threshold: float = 0.75, birch_n_clusters: Union[int, NoneType] = None, dbscan_eps: float = 1.0, min_cluster_size: int = 1, prune_duplicates: bool = True, pruning_threshold: float = 0.85) -> None
 |  
 |  Multi-document summarization. A key idea is to decay similarity values with sentence distance, then apply clustering.
 |  
 |  Args:
 |      tokenizer: A text tokenizer.
 |      similarity_method: One of 'tfidf_cosine', 'jaccard_index', 'normalized_cooccurrence'.
 |      similarity_decay_window: Any two sentences with further distance than this value will have 0 similarity.
 |      similarity_decay_alpha: An exponent for a sentence d

In [101]:
help(lexrank.probe)

Help on method probe in module lexrankr.lexrankr:

probe(k: Union[int, float] = 0) -> List[str] method of lexrankr.lexrankr.LexRank instance



In [102]:
df1['lexrank2'] = np.nan

for i,j in enumerate(df1['본문']):
    try:
        lexrank.summarize(j)
        summaries: List[str] = lexrank.probe(k=0.4) #ratio 0.4
        df1.loc[i, 'lexrank2'] = ' '.join(summaries)
    except ValueError:
        text = j.replace("\n", " ")
        lexrank.summarize(text)
        summaries: List[str] = lexrank.probe(k=0.4)
        df1.loc[i, 'lexrank2'] = ' '.join(summaries)

In [106]:
df1.isna().sum()

리뷰자          0
본문           0
textrank1    0
textrank2    0
lexrank1     0
lexrank2     0
dtype: int64

In [107]:
df1[df1['본문'] == df1['lexrank2']].shape #요약 전후가 동일한 경우 확인

(123, 6)

#### 후처리
- \n 및 특수문자 삭제

In [109]:
# 숫자, 한글, 영어, 공백만 남기기
df1['본문'] = df1['본문'].replace(r'[^0-9가-힣a-zA-Z\s]', '', regex=True)
df1['textrank1'] = df1['textrank1'].replace(r'[^0-9가-힣a-zA-Z\s]', '', regex=True)
df1['textrank2'] = df1['textrank2'].replace(r'[^0-9가-힣a-zA-Z\s]', '', regex=True)
df1['lexrank1'] = df1['lexrank1'].replace(r'[^0-9가-힣a-zA-Z\s]', '', regex=True)
df1['lexrank2'] = df1['lexrank2'].replace(r'[^0-9가-힣a-zA-Z\s]', '', regex=True)

In [114]:
#\n 삭제
df1['본문'] = df1['본문'].replace('\n','',regex=True)
df1['textrank1'] = df1['textrank1'].replace('\n','',regex=True)
df1['textrank2'] = df1['textrank2'].replace('\n','',regex=True)
df1['lexrank1'] = df1['lexrank1'].replace('\n','',regex=True)
df1['lexrank2'] = df1['lexrank2'].replace('\n','',regex=True)

In [116]:
df1.to_csv("data/textrank_lexrank.csv", encoding='utf-8-sig', index=False)